In [2]:
import pandas as pd
import os

In [2]:
import sys
import cx_Oracle
try:
    if sys.platform.startswith("darwin"):
        lib_dir = os.path.join(os.environ.get("HOME"), "Downloads", "instantclient_19_8")
        cx_Oracle.init_oracle_client(lib_dir=lib_dir)
    elif sys.platform.startswith("win32"):
        lib_dir = r"C:\oracle\instantclient_21_6"
        cx_Oracle.init_oracle_client(lib_dir=lib_dir)
except Exception as err:
    print("Whoops!")
    print(err);
    sys.exit(1);

In [3]:
from db_worker import DB_Worker
from utils import parse_config

In [4]:
config = parse_config('../data/config.ini')

In [5]:
config._sections['db']

{'host': '192.168.238.240',
 'port': '1521',
 'dbname': 'study',
 'user': 'sqlbot',
 'pwd': 'iddqd'}

In [6]:
dbw = DB_Worker(config)

In [7]:
tables = [
    'PYTHON_BOT_PROGRESS',
    'PYTHON_BOT_ASSIGNMENTS',
    'PYTHON_BOT_USERS',
    'PYTHON_BOT_GROUPS',
]

In [8]:
# for t in tables:
#     q = f'''
#     select dbms_metadata.get_ddl('TABLE', '{t}') from dual
#     '''
#     c = dbw.connect_to_db()
#     cur = c.cursor()
#     cur.execute(q)
#     out = cur.fetchall()
#     q = out[0][0].read()
#     c.close()
#     print('=============================')
#     print(q)

In [9]:
q = '''
drop table {t}
'''
for t in tables:
    qq = q.format(t=t)
    with dbw.connect_to_db() as c:
        cur = c.cursor()
        try:
            cur.execute(qq)
        except Exception as e:
            print(e)

In [10]:
q = '''
CREATE TABLE PYTHON_BOT_GROUPS (
    "ID" NUMBER NOT NULL ENABLE,
    "NAME" VARCHAR2(100) NOT NULL ENABLE,
    CONSTRAINT "PYTHON_GROUPS_PK" PRIMARY KEY ("ID")
);

CREATE TABLE PYTHON_BOT_USERS (
    "NAME" VARCHAR2(150), 
    "GROUP_ID" NUMBER, 
    "CHAT_ID" VARCHAR2(30), 
    "IS_TEST" CHAR(1) DEFAULT 'N' NOT NULL ENABLE, 
    "LOGIN" VARCHAR2(100), 
    CONSTRAINT "PYTHON_PK_USERS_CHAT_ID" PRIMARY KEY ("CHAT_ID"),
    CONSTRAINT "PYTHON_NAME_NOT_NULL" CHECK ("NAME" IS NOT NULL) ENABLE,
    CONSTRAINT "PYTHON_FK_GROUP_ID" FOREIGN KEY ("GROUP_ID") REFERENCES PYTHON_BOT_GROUPS ("ID") ENABLE
);

CREATE TABLE PYTHON_BOT_ASSIGNMENTS (
    "TASK" VARCHAR(50),
    CONSTRAINT "PYTHON_UK_TEST_TASK" UNIQUE ("TASK")
);

CREATE TABLE PYTHON_BOT_PROGRESS (
    "NAME" VARCHAR2(150),
    "CHAT_ID" VARCHAR2(30),
    "TASK" VARCHAR(50),
    "GROUP_ID" NUMBER,
    "DATE_TIME" DATE DEFAULT sysdate NOT NULL ENABLE,
    CONSTRAINT "PYTHON_FK_CHAT_ID" FOREIGN KEY ("CHAT_ID") REFERENCES PYTHON_BOT_USERS ("CHAT_ID") ENABLE,
    CONSTRAINT "PYTHON_FK_TASK" FOREIGN KEY ("TASK") REFERENCES PYTHON_BOT_ASSIGNMENTS ("TASK") ENABLE
);'''
for qq in q.split(';'):
    if qq=='': continue
    with dbw.connect_to_db() as c:
        cur = c.cursor()
        cur.execute(qq)
        # out = cur.fetchall()

In [4]:
from hw_tests_prod.utils import _check_task_name_format
tests = os.listdir(os.path.join(os.path.abspath(''), 'hw_tests_prod'))
tests_names = []
for t in tests:
    if t[:5] == 'test_':
        tname, ext = os.path.splitext(t[5:])
        assert ext == '.py'
        try:
            _check_task_name_format(tname)
            tests_names.append(tname)
        except Exception:
            pass

In [12]:
q = '''
delete from PYTHON_BOT_ASSIGNMENTS where 1=1
'''
with dbw.connect_to_db() as c:
    cur = c.cursor()
    cur.execute(q)
    c.commit()
q = 'insert all\n'
for t in tests_names:
    q += f"into PYTHON_BOT_ASSIGNMENTS (TASK) values ('{t}')\n"
q += 'select 1 from dual'
with dbw.connect_to_db() as c:
    cur = c.cursor()
    cur.execute(q)
    c.commit()

In [13]:
q = '''
select * from PYTHON_BOT_ASSIGNMENTS
'''
with dbw.connect_to_db() as c:
    df = pd.read_sql(q, c)
df

,TASK
0,L04_HW_project
1,L04_HW_task1
2,L04_HW_task2
3,L04_HW_task3
4,L05_HW_project
5,L06_HW_project
6,L06_HW_task1
7,L06_HW_task2


In [14]:
q = '''
insert into PYTHON_BOT_GROUPS (id, name)
values (1,'admin')
'''
with dbw.connect_to_db() as c:
    cur = c.cursor()
    cur.execute(q)
    c.commit()
    # out = cur.fetchall()
# out

In [39]:
q = '''
INSERT INTO PYTHON_BOT_PROGRESS (name, group_id, chat_id, task)
VALUES ('Nikita Kakurnikov', 1, '258477531', 'L04_HW_task1')
'''
with dbw.connect_to_db() as c:
    cur = c.cursor()
    cur.execute(q)
    c.commit()
    # out = cur.fetchall()
# out

IntegrityError: ORA-02291: integrity constraint (SQLBOT.PYTHON_FK_TASK) violated - parent key not found

In [40]:
q = '''
select * from PYTHON_BOT_ASSIGNMENTS
'''
with dbw.connect_to_db() as c:
    df = pd.read_sql(q, c)
df

,TASK
0,L04_project
1,L04_task1
2,L04_task2
3,L04_task3
4,L05_project
5,L05_task1


In [35]:
q = '''
select * from PYTHON_BOT_GROUPS
'''
with dbw.connect_to_db() as c:
    df = pd.read_sql(q, c)
df

,ID,NAME
0,1,test


In [38]:
q = '''
select * from PYTHON_BOT_PROGRESS
'''
with dbw.connect_to_db() as c:
    df = pd.read_sql(q, c)
df

,NAME,CHAT_ID,TASK,GROUP_ID,DATE_TIME


In [ ]:
q = '''
select
task,
count(*) as n_done
from (
    select
    chat_id, task
    from python_bot_progress
    group by
    chat_id, task
) t
group by
task
order by
task
'''